# Import basic libraries

Let's import some basic stuff

### Fix for `fiona`

*`fiona` has an issue with GDAL 3.0. Better set your `GDAL_DATA` path to fiona's installation (contains GDAL 2.4.4) prior to running this script.*

In [3]:
import geopandas
import osmnx as ox

# Load the OSM data

This will load the osm file to a network

In [4]:
roads_graph = ox.graph_from_file('../data/Breda/Breda extract.osm', simplify=False)
edges = ox.graph_to_gdfs(roads_graph, nodes=False, node_geometry=False)

edges = edges.to_crs("EPSG:28992")

Investigate

In [5]:
edges.head()

,u,v,key,osmid,name,highway,area,oneway,length,geometry,maxspeed,landuse,service,bridge,access,tunnel,lanes,ref,width
0,689995792,689990659,0,126559191,Havermarkt,pedestrian,yes,False,5.549,"LINESTRING (112471.505 400203.536, 112466.591 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,689995792,689990659,1,267783561,NaN,NaN,NaN,False,5.549,"LINESTRING (112471.505 400203.536, 112466.591 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,689995792,2731861671,0,126559191,Havermarkt,pedestrian,yes,False,5.977,"LINESTRING (112471.505 400203.536, 112477.359 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,689995792,2731861671,1,267778236,NaN,NaN,NaN,False,5.977,"LINESTRING (112471.505 400203.536, 112477.359 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,689995792,2731859065,0,267778236,NaN,NaN,NaN,False,14.574,"LINESTRING (112471.505 400203.536, 112472.478 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Load the BGT data

In [6]:
bgt_roads = geopandas.read_file('../data/Breda/bgt_roads.geojson')

len(bgt_roads)

38859

## TODO

- Use `touch` operation to assign any "child" bgt polygon to the corresponding road (e.g. a `voetpad` that touches a `weg` should take its `gml_id` as parent).
- Group parent and its children as semantic surfaces of the same road.

In [46]:
pd.unique(bgt_roads['geometry'].boundary.type)

array(['LineString', 'MultiLineString'], dtype=object)

# Intersect roads network with polygon
First, we intersect the road lines with the BGT polygons (to create nodes at the polygon boundaries):

In [7]:
roads = geopandas.overlay(edges, bgt_roads, how='intersection')

Then, dissolve the roads by `gml_id` (just to group lines per BGT polygon):

In [8]:
roads_dissolved = roads.dissolve(by='gml_id')

Export the intersected roads to `GeoJSON`:

In [10]:
roads_dissolved.to_file('output/roads_dissolved.geojson', driver='GeoJSON')

# Export to CityJSON

Let's export everything to CityJSON.

First, we'll define how the established intersected lines will be translated to `Road` objects:

In [50]:
def get_id(feature):
    if isinstance(feature['osmid'], list):
        return str(feature['osmid'][0])
    else:
        return str(feature['osmid'])

def process_line(line, vertices):
    points = [[x, y, 0] for x, y in list(line.coords)]
    indices = [i + len(vertices) for i in range(len(points))]
    for p in points:
        vertices.append(p)
    
    return indices

def process_geometry(geom, vertices):
    if geom.type == "LineString":
        indices = [ process_line(geom, vertices) ]
    elif geom.type == "MultiLineString":
        indices = []
        for line in geom.geoms:
            indices.append(process_line(line, vertices))
    elif geom.type == "Polygon":
        indices = [process_geometry(geom.boundary, vertices)]
    
    return indices

def create_cityobject(feature, vertices):
    indices = process_geometry(feature['geometry'], vertices)

    return {
        "type": "Road",
        "attributes": {
            "osm_id": feature['osmid']
        },
        "geometry": [
            {
                "type": "MultiLineString",
                "lod": "0.1",
                "boundaries": indices
            }
        ]
    }

Now, let's run this against all intersected road segments:

In [51]:
vertices = []

objects = {i: create_cityobject(f, vertices) for i, f in roads_dissolved.iterrows()}

for i, f in bgt_roads.iterrows():
    if f['gml_id'] in objects:
        objects[f['gml_id']]['geometry'].append({
            "type": "MultiSurface",
            "lod": "2",
            "boundaries": process_geometry(f['geometry'], vertices)
        })

Finally, let's export everything as CityJSON:

In [52]:
import json
import io

output = {
  "type": "CityJSON",
  "version": "1.0",
  "CityObjects": objects,
  "vertices": vertices
}

with open('output/breda.json', 'w') as file:
    json.dump(output, file)